
# 6. Библиотеки для получения и обработки данных

## 6.3. Модуль requests

### Теория

**OSI (The Open Systems Interconnection model, модель взаимодействия открытых систем)** - стандарт, определяющий уровни передачи данных.

**Протокол** - это набор правил, который определяет процесс обмена данными между различными устройствами или программами.

**HTTP (HyperText Transfer Protocol, протокол передачи гипертекста)** - один из самых популярных протоколов для обмена данными между программами в компьютерных сетях (прикладного уровня):
- Код ответа **200** - ответ HTTP-сервера, значит, что запрос был обработан успешно;
- Код ответа **404** - запрошенный объект (чаще всего веб-страница) не был найден

В протоколе `HTTP` описаны различные виды запросов: на получение данных (`GET`), на передачу данных (`POST`) и др.


**API (Application Programming Interface)** - набор правил для взаимодействия с сервисом (протокол сервиса, действующий над протоколом HTTP).

#### API сервиса Яндекс.Карт на примере Static API

Static API  возвращает изображение карты в ответ на HTTPS-запрос. Документация: https://yandex.ru/dev/maps/staticapi/doc/1.x/dg/concepts/about.html. Запрос для браузера имеет следующий вид: https://static-maps.yandex.ru/1.x/?{параметры URL}

Введем запрос в браузере:
> https://static-maps.yandex.ru/1.x/?ll=37.677751,55.757718&spn=0.016457,0.00619&l=map

Параметры:
- `II` - координаты центра карты (через запятую - долгота и широта в градусах);
- `spn` - область показа (протяженность карты в градусах по долготе и широте);
- `l` - тип карты (в запросе используется тип map – схема).

#### requests

`requests` - библиотека для удобного формирования HTTP-запросов и получения ответов (не стандартная, устанавливается через `pip install requests`)

In [4]:
# vibo: импортируем функцию get()
from requests import get

# vibo: формируем тот же запрос, что и выше через функцию get() библиотеки requests
response = get("https://static-maps.yandex.ru/1.x/?"
             "ll=37.677751,55.757718&"
             "spn=0.016457,0.00619&"
             "l=map")
print(response) # <Response [200]> ответ HTTP-сервера, значит, запрос был обработан успешно

# vibo: или так

# vibo: можно переделать запись, используя аргумент params (словарь)
params = {"ll": "37.677751,55.757718",
          "spn": "0.016457,0.00619",
          "l": "map"}
response = get("https://static-maps.yandex.ru/1.x/", params=params)

<Response [200]>


In [3]:
# vibo: для получения данных из ответа используем атрибут content, это данные в виде последовательности байтов,
# vibo: в соответствии с документацией Static API, ответ - файл в формате `PNG`

# vibo: запишем данные в файл
# vibo: `wb` - запись в бинарном режиме
with open("map.png", "wb") as file:
    file.write(response.content)

Тот же запрос с обработкой исключения на потерю соединения (ConnectionError)

In [6]:
# vibo: проверка подключения к сети, исключение ConnectionError
from requests import get, ConnectionError

params = {"ll": "37.677751,55.757718",
          "spn": "0.016457,0.00619",
          "l": "map"}
try:
    response = get("https://static-maps.yandex.ru/1.x/", params=params)
except ConnectionError:
    print("Проверьте подключение к сети.")
# vibo: если не вызвано исключение в блоке try
else:
    with open("map.png", "wb") as file:
        file.write(response.content)

#### API. Яндекс.Диск

Документация: https://yandex.ru/dev/disk/rest/. Для работы с API Яндекс.Диска требуется авторизация пользователя по протоколу **OAuth 2.0.**

Открытый протокол авторизации **OAuth 2.0** обеспечивает предоставление третьей стороне ограниченный доступ к защищённым ресурсам пользователя без передачи ей логина и пароля. Вместо логина и пароля авторизация производится по **OAuth-токену**. **Токен** – это уникальная для каждого пользователя строка-ключ для доступа с помощью API к файлам в облаке.

Для работы с API необходимо создать и зарегистрировать приложение-сервис по инструкции: https://yandex.ru/dev/id/doc/dg/oauth/tasks/register-client.html

Ссылка для создания сервиса: https://oauth.yandex.ru/client/new, заполняем:
- Заполнить поле "Название сервиса".
- Выбрать вариант платформы "Веб-сервисы".
- Выбрать значение "Подставить URL для разработки" в поле Callback URL.
- В качестве доступных сервису данных указать "Яндекс.Диск REST API" и выдать все четыре возможных права доступа.
- Нажать кнопку "Создать приложение".

После успешной регистрации получаем значения для получения токена по протоколу авторизации OAuth 2.0:
- **ClientID** - это идентификатор вашего приложения; используйте его в запросах для получения OAuth-токена;
- **Client secret** - секретный ключ, которым будет подписан jwt-токен с информацией о пользователе;
- **Callback URI (Redirect URL)** - на этот адрес направим пользователя после авторизации;

**Протокол OAuth 2.0 реализован в библиотеке requests-oauthlib**

In [ ]:
!pip install requests requests_oauthlib

Приложение-сервис должно перенаправить пользователя по ссылке для авторизации. Пользователь переходит по ссылке и копирует сгенерированный одноразовый код, а затем вводит его в приложение. Библиотека `requests-oauthlib` на основе введённого кода делает запрос на получение токена и возвращает его. Более детальный порядок получения токена в инструкции: https://yandex.ru/dev/id/doc/dg/oauth/reference/auto-code-client.html#auto-code-client__get-code

In [ ]:
# vibo: напишем программу с авторизацией сервиса и получением токена

from requests_oauthlib import OAuth2Session
from requests import get, post, put, delete


# vibo: client_id и client_secret - получаем после регистрации приложения
client_id = ""
client_secret = ""
auth_url = "https://oauth.yandex.ru/authorize"
token_url = "https://oauth.yandex.ru/token"
oauth = OAuth2Session(client_id=client_id)
authorization_url, state = oauth.authorization_url(auth_url, force_confirm="true")
print("Перейдите по ссылке, авторизуйтесь и скопируйте код:", authorization_url)
code = input("Вставьте одноразовый код: ")
token = oauth.fetch_token(token_url=token_url,
                          code=code,
                          client_secret=client_secret)
# vibo: переменная token является словарем, в котором по ключу access_token находится нужный токен
access_token = token["access_token"]
print(f'access_token: {access_token}')

In [15]:
# vibo: создадим словарь headers
# vibo: полученный на этапе авторизации токен необходимо передавать во всех запросах к API
# # vibo: для этого нужно заполнять в заголовке запроса поле Authorization значением OAuth <ваш токен>
headers = {"Authorization": f"OAuth {access_token}"}
# vibo: выполним GET-запрос по адресу https://cloud-api.yandex.net/v1/disk для получения информации о состоянии облачного хранилища
r = get("https://cloud-api.yandex.net/v1/disk", headers=headers)
#vibo: данные в ответе имеют формат json, преобразуются в словарь методом json()
print(r.json())

{'max_file_size': 1073741824, 'paid_max_file_size': 53687091200, 'total_space': 11811160064, 'trash_size': 0, 'is_paid': False, 'used_space': 399077229, 'system_folders': {'odnoklassniki': 'disk:/Социальные сети/Одноклассники', 'google': 'disk:/Социальные сети/Google+', 'instagram': 'disk:/Социальные сети/Instagram', 'vkontakte': 'disk:/Социальные сети/ВКонтакте', 'attach': 'disk:/Почтовые вложения', 'mailru': 'disk:/Социальные сети/Мой Мир', 'downloads': 'disk:/Загрузки/', 'applications': 'disk:/Приложения', 'facebook': 'disk:/Социальные сети/Facebook', 'social': 'disk:/Социальные сети/', 'messenger': 'disk:/Файлы Мессенджера', 'calendar': 'disk:/Материалы встреч', 'scans': 'disk:/Сканы', 'screenshots': 'disk:/Скриншоты/', 'photostream': 'disk:/Фотокамера/'}, 'user': {'country': 'ru', 'login': 'vv.bo', 'display_name': 'vibo', 'uid': '250371050'}, 'unlimited_autoupload_enabled': False, 'revision': 1668703522800636}


Все возможные запросы к сервису, их формат и ответы сервера можно посмотреть в документации: https://yandex.ru/dev/disk/poligon/

In [18]:
# vibo: 1. создаем на яндекс диске новую папку "Тест API"

# vibo: полученный на этапе авторизации токен необходимо передавать во всех запросах к API (headers)
# vibo: для этого нужно заполнять в заголовке запроса поле Authorization значением OAuth <ваш токен>
# vibo: в params передаем путь
params = {"path": "Тест API"}
r = put("https://cloud-api.yandex.net/v1/disk/resources", headers=headers, params=params)
print(r) # Response [201]

<Response [201]>


In [20]:
# vibo: копируем файл в папку на яндекс диск

params = {"path": "Тест API/map.png"}
# vibo: выполняем GET-запроc по адресу https://cloud-api.yandex.net/v1/disk/resources/upload
# дял получения ссылки на загрузку в облако
r = get("https://cloud-api.yandex.net/v1/disk/resources/upload",
        headers=headers, params=params)
# vibo: адрес в поле href ответа сервера
href = r.json()["href"]
# vibo: открываем файл в режиме бинарного чтения rb
files = {"file": open("map.png", "rb")}
# vibo: PUT-запрос на запись файла
# vibo: Передавать токен в данном запросе не нужно
r = put(href, files=files)
print(r) # Response [201]

<Response [201]>


### Практика /10

In [ ]:
# A ПОЛНОЕ РЕШЕНИЕ
# vibo: вывод ответа сервера
from requests import get


response = get("http://127.0.0.1:5000/")
print(response.text)

In [ ]:
# B ПОЛНОЕ РЕШЕНИЕ
# vibo: суммирование ответов сервера
from requests import get


# vibo: получаем от пользователя адрес
link = f'http://{input()}'
# vibo: запрашиваем у сервера данные
data = int(get(link).text)

ans = 0
# vibo: пока ответ сервера не равен 0 суммируем
while data != 0:
    ans += data
    # vibo: снова запрашиваем данные
    data = int(get(link).text)

print(ans)

In [11]:
# C ПОЛНОЕ РЕШЕНИЕ
from requests import get


# vibo: получаем от пользователя адрес
link = f'http://{input()}'
# vibo: запрашиваем у сервера данные (ответ в json по условию)
data = get(link).json()

# data = [1, 2, "ошибка", 3]
# data = [7, 3, ["ошибка"], -5, 2]

print(sum([x for x in data if type(x) == int]))

6


In [16]:
# # D ПОЛНОЕ РЕШЕНИЕ
# from requests import get
#
#
# link = f'http://{input()}'
# key_name = input()
#
# data = get(link).json()
#
# # data = {"first": "1", "second": "2"}
# # data = {"first": "1", "third": "3"}
#
# try:
#     print(data[key_name])
# except KeyError:
#     print("No data")

No data


In [ ]:
# D ПОЛНОЕ РЕШЕНИЕ
# vibo: проще без try/except
from requests import get


link = f'http://{input()}'
key_name = input()

response = get(link).json()

print(response.get(key_name, 'No data'))

In [ ]:
# # E ПОЛНОЕ РЕШЕНИЕ
# from sys import stdin
# from requests import get
#
#
# link = f"http://{input()}"
# ans = 0
# for line in stdin:
#     # vibo: rstrip("\n") - убираем символ переноса на новую строку
#     way = line.rstrip("\n")
#     ans += sum(get(f'{link}{way}').json())
#
# print(ans)

In [ ]:
# E ПОЛНОЕ РЕШЕНИЕ
# vibo: можно заменить stdin на stdin.read().splitlines()
# vibo: тогда не нужно отрезать \n

from sys import stdin
from requests import get


link = f"http://{input()}"
ans = 0
for way in stdin.read().splitlines():
    ans += sum(get(link + way).json())

print(ans)